# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather = pd.read_csv("My_Weather.csv")
weather.head()


,City,Country,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Date
0,cairns,AU,-16.92,145.77,80.60,78,75,2.80,1608564174
1,bredasdorp,ZA,-34.53,20.04,69.80,83,90,13.87,1608564182
2,iqaluit,CA,63.75,-68.51,10.40,78,90,12.75,1608563971
3,port alfred,ZA,-33.59,26.89,70.00,98,100,4.00,1608564183
4,sola,VU,-13.88,167.55,80.56,82,3,13.02,1608564183


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
lat_lng = weather[['Lat', 'Lng']]

humid = weather["Humidity"]

lat_lng.head()

,Lat,Lng
0,-16.92,145.77
1,-34.53,20.04
2,63.75,-68.51
3,-33.59,26.89
4,-13.88,167.55


In [7]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(lat_lng, weights = humid)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
ideal_temp = (weather.Temp < 80) & (weather.Temp > 70)

ideal_ws = weather["Wind Speed"] < 15
ideal_clouds = weather.Cloudiness == 0
ideal = ideal_temp & ideal_ws & ideal_clouds 

dream_vacay = weather[ideal]
dream_vacay = dream_vacay.dropna()
dream_vacay = dream_vacay.reset_index()
dream_vacay.head(10)

,index,City,Country,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Date
0,101,sechura,PE,-5.56,-80.82,75.72,62,0,9.48,1608564199
1,104,tucuman,AR,-26.82,-65.22,77.00,50,0,5.08,1608564054
2,113,viedma,AR,-40.81,-63.00,75.99,38,0,7.00,1608564201
3,190,atuona,PF,-9.80,-139.03,77.97,74,0,14.88,1608564214
4,237,sur,OM,22.57,59.53,72.50,69,0,9.93,1608564222
5,312,acajutla,SV,13.59,-89.83,77.00,65,0,5.82,1608564234
6,323,general roca,AR,-39.03,-67.58,78.80,36,0,6.93,1608564235
7,349,dwarka,IN,22.24,68.97,71.47,54,0,11.21,1608564239
8,394,umm lajj,SA,25.02,37.27,77.70,58,0,5.41,1608564246
9,463,tiznit,MA,29.58,-9.50,70.11,28,0,3.00,1608564257


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = dream_vacay
# Add column for Hotel Name - Note that we used "" to specify initial entry.
hotel_df['Hotel Name'] = ""

In [21]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

for index, row in hotel.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    try:
        hotel.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Missing field/result... skipping.
Missing field/result... skipping.


,index,City,Country,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
0,101,sechura,PE,-5.56,-80.82,75.72,62,0,9.48,1608564199,Hotel Loma Dorada
1,104,tucuman,AR,-26.82,-65.22,77.00,50,0,5.08,1608564054,HOTEL PORTAL DEL NORTE
2,113,viedma,AR,-40.81,-63.00,75.99,38,0,7.00,1608564201,Hotel Austral Viedma
3,190,atuona,PF,-9.80,-139.03,77.97,74,0,14.88,1608564214,CHALETS HANAKEE
4,237,sur,OM,22.57,59.53,72.50,69,0,9.93,1608564222,Sur Grand Hotel


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…